In [ ]:
import pandas as pd 
df= pd.read_csv(r"C:\Users\johnn\Desktop\internship\ecommerce_furniture_dataset_2024.csv")
print(df.info())
print(df[['originalPrice', 'price']].head(10))
print(df['originalPrice'].isna().sum(), "missing originalPrice values")

In [ ]:
# remove the '$', ',' and convert to floating point 
df['price'] = df['price'].replace(r'[\$,]', "", regex= True).astype(float)
df['originalPrice']= df['originalPrice'].replace(r'[\$,]', "", regex=True)

#original to float ,and having the nan value
df['originalPrice']= pd.to_numeric(df['originalPrice'], errors= 'coerce')

#filling the missing values 
df['originalPrice'].fillna(df['price'], inplace=True)
print(df[['originalPrice', 'price']].head(10))

In [ ]:
df['discount']= ((df['originalPrice'] - df['price'])/ df['originalPrice'])*100
print(df[['originalPrice', 'price', 'discount']]. head(10))

In [ ]:
#discount flag and the discount bucket 
df['is_discount']= (df['discount']> 0).astype(int)
df['discount_level']= pd.cut(df['discount'], bins=[-1,0,25,50,75,100], labels=['No Discount', '0-25%','25-50%','50-75%','75-100%'])

In [ ]:
# EDA
import matplotlib.pyplot as plt 
print("Average Discount%:", df['discount'].mean())
print(df['is_discount'].value_counts())

plt.hist(df['discount'], bins=20, edgecolor='Black')
plt.xlabel('Discount %')
plt.ylabel("Nnmber of Product")
plt.title("Distribution of Discounts")
plt.show()

In [ ]:
print(df.groupby('is_discount')['price'].mean())

In [ ]:
# loading data to SQl
from sqlalchemy import create_engine
import pandas as pd 
import urllib.parse

username='root'
password= urllib.parse.quote_plus('#john123@project')
host='localhost'
port= 3306
database ='ecommerce'

engine= create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")

df.to_sql("Product_clean", engine, if_exists= "replace", index= False)
print("Data loaded successfully!!")

In [ ]:
# NOW TRAING OF MODDEL IS BELOW 

In [ ]:
# TRAINING MODEL 

In [8]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import joblib
import urllib.parse
from sqlalchemy import create_engine 


username = "root"
password= urllib.parse.quote_plus("#john123@project")
host="localhost"
port= 3306
database="ecommerce"
engine= create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")

df = pd.read_sql("select * from ml_mod" , engine)
print("Data Shape:", df.shape)
print(df.head(10))
print("sql loaded successfully from view!")

Data Shape: (2000, 16)
                                        productTitle  originalPrice   price  \
0  Dresser For Bedroom With 9 Fabric Drawers Ward...          46.79   46.79   
1  Outdoor Conversation Set 4 Pieces Patio Furnit...         169.72  169.72   
2  Desser For Bedroom With 7 Fabric Drawers Organ...          78.40   39.46   
3  Modern Accent Boucle Chair,Upholstered Tufted ...         111.99  111.99   
4  Small Unit Simple Computer Desk Household Wood...          48.82   21.37   
5  3 Pieces Patio Furniture Set, Outdoor Swivel G...        1294.04  123.40   
6  5-Piece Patio Furniture Set Outdoor Couch with...         198.31  198.31   
7  Living Room Furniture, Modern 3-Piece Includin...        1474.17  734.58   
8  TV Stand Dresser For Bedroom With 5 Fabric Dra...          43.96   43.96   
9  Outdoor Furniture Set 3 Pieces Wicker Patio Fu...         233.30  233.30   

   sold   discount  is_discount  discount_level_num  price_diff  \
0   600   0.000000            0         

In [13]:
X= df.drop(columns=['sold','productTitle'])
y= df['sold']

In [14]:
X_train , X_test, y_train, y_test = train_test_split(X,y, test_size=0.25, random_state= 42)

In [15]:
rf= RandomForestRegressor(random_state=42, n_jobs= -1)

param_grid={
    "n_estimators":[100,200],
    "max_depth":[10,20,None],
    "min_samples_split":[2,5],
    "min_samples_leaf":[1,2]
}

grid_search = GridSearchCV(
    rf, param_grid,
    cv=3, scoring="neg_mean_squared_error",
    verbose=2, n_jobs=-1
)

grid_search.fit(X_train, y_train)

print("Best Parameters:", grid_search.best_params_)

Fitting 3 folds for each of 24 candidates, totalling 72 fits
Best Parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}


In [17]:
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2= r2_score(y_test, y_pred)
print("RMSE:", rmse)
print("R^2 Score:", r2)

RMSE: 31.07375761904914
R^2 Score: 0.9593216802325009


In [19]:
joblib.dump(best_model,"random_forest_model.pkl")
print("Model saved as random_forest_model.pkl")

Model saved as random_forest_model.pkl


In [21]:
results= pd.DataFrame({
    "Actual": y_test,
    "Predicted": y_pred
})

results.to_csv("rf_predictons.csv", index=False)
print("Pridiction save to rf_predictions.csv")

Pridiction save to rf_predictions.csv


In [22]:
import os 

curr_dir= os.getcwd()
print(curr_dir)

C:\Users\johnn


In [21]:
# Save predictions to MySQL (offline)
import joblib, pandas as pd, urllib.parse
from sqlalchemy import create_engine

# config
username = "root"
password = urllib.parse.quote_plus("#john123@project")
host = "localhost"
port = 3306
database = "ecommerce"
engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}")

# load feature view (same one you trained on)
df = pd.read_sql("SELECT * FROM ml_mod", engine)

# prepare features exactly as your model expects
X = df.drop(columns=["productTitle", "sold"])   # adjust if your model used other cols
model = joblib.load(r"C:\Users\johnn\random_forest_model.pkl")  # update path

# predict
preds = model.predict(X)
# if model was trained on log1p target, invert:
# preds = np.expm1(preds)

# assemble predictions table
pred_df = pd.DataFrame({
    "productTitle": df["productTitle"],
    "predicted_sold": preds
})

# save to MySQL (replace table if exists)
pred_df.to_sql("product_predictions", engine, if_exists="replace", index=False, chunksize=1000)
print("Saved product_predictions to MySQL")


Saved product_predictions to MySQL
